<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/%E8%87%AA%E5%AE%9A%E4%B9%89%E5%BE%AA%E7%8E%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#标准开场
%tensorflow_version 2.x
import tensorflow as tf
print('tensorflow:',format(tf.__version__))
import numpy as np
from tensorflow import keras
import glob
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

TensorFlow 2.x selected.
tensorflow: 2.1.0


In [3]:
#引入minist数据集
(train_image,train_label),(test_image,test_label)=tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
#给图像增加维度
train_image=tf.expand_dims(train_image,-1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [0]:
#预处理，改变数据类型
train_image=tf.cast(train_image/255,tf.float32)

In [0]:
#预处理，转换label的数据类型
train_label=tf.cast(train_label,tf.int64)

In [0]:
#把它作为tf.data加载进来,建立dataset
dataset=tf.data.Dataset.from_tensor_slices((train_image,train_label))

In [8]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [0]:
#对数据进行乱序，这里对10000个数据进行乱序
#批次的大小,用batch
dataset=dataset.shuffle(10000).batch(32)
#这里没有设置repeat

In [0]:
#建立模型，进行自定义训练
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(28,28,1)))
model.add(tf.keras.layers.Conv2D(32,[3,3],activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [11]:
#看一下model的可训练参数
model.trainable_variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32, numpy=
 array([[[[ 1.24206588e-01,  1.97288990e-02,  4.11526859e-02,
            9.23781842e-02,  1.20780870e-01,  1.18823126e-01,
            5.96801192e-02,  4.46898639e-02,  1.54365733e-01,
           -2.43997276e-02, -1.31914258e-01,  2.23230720e-02,
            1.53205842e-02, -3.41444463e-02, -1.67575255e-01,
            1.57724485e-01]],
 
         [[-1.62954003e-01,  6.81052655e-02, -1.25282735e-01,
           -1.13688931e-01, -1.31640643e-01,  9.18239206e-02,
           -9.63260606e-02, -1.86701417e-01,  2.81471908e-02,
            1.35003522e-01, -1.61264241e-02, -1.73140079e-01,
           -4.24175411e-02, -3.41285318e-02, -1.79138467e-01,
           -1.92309067e-01]],
 
         [[-7.93241858e-02,  6.02493435e-02,  1.06759027e-01,
           -1.89887837e-01, -9.38427970e-02, -6.10521287e-02,
           -1.09910220e-02,  1.18955269e-01, -1.07690923e-01,
            1.93863407e-01,  1.57023713e-01, -9.29432809

优化器

In [0]:
optimizer=tf.keras.optimizers.Adam()

自定义 损失函数

In [0]:
loss_func=tf.keras.losses.SparseCategoricalCrossentropy()

In [0]:
#设计一个生成器
features,labels=next(iter(dataset))

In [15]:
features.shape

TensorShape([32, 28, 28, 1])

In [16]:
predictions=model(features)
predictions.shape

TensorShape([32, 10])

In [17]:
#哪个最大呢？
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 8, 9, 9, 9, 9, 9])>

In [0]:
#开始训练
#定义一个loss函数
def loss(model,x,y):#x是实际的数据，y是实际的label
  y_=model(x)#y_是预测的label
  return loss_func(y,y_)
#这样的话 我们输入一个model和数据还有标签就会返回一个loss损失函数

In [0]:
#一步优化过程
def train_step(model,images,labels):
  with tf.GradientTape() as t:
    #计算损失函数
    loss_step=loss(model,images,labels)
  grads=t.gradient(loss_step,model.trainable_variables)
  #优化grads 使梯度下降最快
  optimizer.apply_gradients(zip(grads,model.trainable_variables))
  #这里就是应用这个梯度对可训练参数进行优化
  #以上就完成了对一个batch数据的优化

In [0]:
def train():#定义多层训练函数
  for epoch in range(10):
    #然后我们就可以对dataset进行迭代
    for (batch,(trian_image,trian_label)) in enumerate(dataset):
      #这里使用了enumerate枚举的方法
      train_step(model,train_image,trian_label)
      #这样的话就进行了一次训练
    print('Epoch{} is finshed',format(batch))

In [21]:
train()

ValueError: ignored